In [1]:
%matplotlib inline
import sys, os, shutil
from glob import glob
import numpy as np
from utils import *
from vgg16 import Vgg16
from PIL import Image
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from IPython.display import FileLink

Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:04.0)
Using Theano backend.
/home/akshay/anaconda2/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DATA_DIR = '/home/akshay/workspace/data/redux/'
val_path = DATA_DIR + 'valid/'
train_path = DATA_DIR + 'train/'
test_path = DATA_DIR + 'test/'
results_path = DATA_DIR+ 'results/'

In [3]:
g = glob(train_path + '*.jpg')
g = np.random.permutation(g)
for i in range(2000):
    shutil.move(g[i], val_path)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
g_cat = glob(train_path+'cat*.jpg')
g_dog = glob(train_path+'dog*.jpg')
for img in g_cat:
    shutil.move(img, train_path+'cats/')
for img in g_dog:
    shutil.move(img, train_path+'dogs/')

g_cat = glob(val_path+'cat*.jpg')
g_dog = glob(val_path+'dog*.jpg')
for img in g_cat:
    shutil.move(img, val_path+'cats/')
for img in g_dog:
    shutil.move(img, val_path+'dogs')



Now that we have organised our data lets finetune VGG16 model. 

## Finetune

In [4]:
batch_size=64
no_epochs=5

In [5]:
vgg = Vgg16()
train_batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(val_path, batch_size=batch_size)
vgg.finetune(train_batches)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
latest_weights_filename = None
for epoch in range(no_epochs):
    print ("\nRunning Epoch :  {0}".format(epoch))
    vgg.fit(train_batches, val_batches, nb_epoch=1)
    latest_weights_filename = "ft{0}.h5".format(epoch)
    vgg.model.save_weights(results_path+latest_weights_filename)
print ("Completed {0} fit operation\n".format(no_epochs))


Running Epoch :  0
Epoch 1/1
23000/23000 [==============================] - 523s - loss: 0.1248 - acc: 0.9676 - val_loss: 0.0461 - val_acc: 0.9865

Running Epoch :  1
Epoch 1/1
23000/23000 [==============================] - 510s - loss: 0.0962 - acc: 0.9772 - val_loss: 0.0674 - val_acc: 0.9835

Running Epoch :  2
Epoch 1/1
23000/23000 [==============================] - 511s - loss: 0.0964 - acc: 0.9788 - val_loss: 0.0592 - val_acc: 0.9850

Running Epoch :  3
Epoch 1/1
23000/23000 [==============================] - 511s - loss: 0.0928 - acc: 0.9795 - val_loss: 0.0639 - val_acc: 0.9855

Running Epoch :  4
Epoch 1/1
23000/23000 [==============================] - 514s - loss: 0.0993 - acc: 0.9800 - val_loss: 0.0696 - val_acc: 0.9835
Completed 5 fit operation



In [ ]:
for epoch in range(no_epochs):
    print ("Epoch {0}".format(epoch))
    vgg.model.load_weights(filepath=results_path+'ft{0}.h5'.format(epoch))
    test_batches, test_preds = vgg.test(test_path, batch_size=batch_size)
    isdog = test_preds[:,1]
    isdog = isdog.clip(min=0.05, max=0.95)
    test_filenames = test_batches.filenames
    test_file_ids = [int(f[8:-4]) for f in test_filenames]
    subm = np.stack([test_file_ids, isdog], axis=1)
    subm_filename = "submission{0}.csv".format(epoch)
    np.savetxt(subm_filename, subm, fmt='%d,%0.5f', header='id,label', comments='')

Epoch 0
Found 12500 images belonging to 1 classes.
Epoch 1
Found 12500 images belonging to 1 classes.
Epoch 2
Found 12500 images belonging to 1 classes.


In [5]:
FileLink('submission3.csv')

/home/akshay/workspace/redux/submission3.csv